# SWAST Forecasting Tool

An ensemble of Regression with ARIMA Errors and Facebook Prophet

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


import warnings
warnings.filterwarnings('ignore')

check statsmodels version

In [2]:
import statsmodels as sm
print(sm.__version__)

import os

os.getcwd()

0.11.1


'/Users/thomasmonks/Documents/code/swast-forecast-tool'

In [3]:
from swast_forecast.utility import (pre_process_daily_data, 
                                    default_ensemble,
                                    forecast, 
                                    multi_region_forecast)

Importing plotly failed. Interactive plots will not work.


## Constants

In [4]:
PATH = '../ambo_data/Daily_Responses_5_Years_2019_full.csv'

## Read in the data

In [5]:
clean = pre_process_daily_data(path=PATH, 
                               observation_col='Actual_Value', 
                               index_col='Actual_dt')
clean.head()

ora,BNSSG,Cornwall,Devon,Dorset,Gloucestershire,OOA,Somerset,Trust,Wiltshire
actual_dt,,,,,,,,,
2013-12-30,415.0,220.0,502.0,336.0,129.0,NaN,183.0,2042.0,255.0
2013-12-31,420.0,236.0,468.0,302.0,128.0,NaN,180.0,1996.0,260.0
2014-01-01,549.0,341.0,566.0,392.0,157.0,NaN,213.0,2570.0,351.0
2014-01-02,450.0,218.0,499.0,301.0,115.0,NaN,167.0,2013.0,258.0
2014-01-03,419.0,229.0,503.0,304.0,135.0,NaN,195.0,2056.0,269.0


## Creating and fitting an Ensemble model to a region

The easy way to create an ensemble model is to call the `default_ensemble()` function form the utility module.  This returns the best known forecasting model.

In [6]:
model = default_ensemble()
model

ProphetARIMAEnsemble(order=(1, 1, 3), seasonal_order=(1, 0, 1, 7), prophet_default_alpha=0.2)

The code above informs us that the ensemble includes a Regression model with ARIMA errors with parameters (1, 1, 3)(1, 0, 1, 7).  By default a Prophet model will create a 80\% prediction interval 100(1-alpha)

To fit we call the `.fit()` method and pass in a `pd.Series` (or `pd.DataFrame`) that contains the historical observations.  By default you do not need to pass in holidays.  The ensemble will model new years day automatically (via Prophet's holidays function and as a dummy variable in the Regression with ARIMA errors).

In [7]:
#example - fitting Wiltshire - this will take a few seconds.
model.fit(clean['Wiltshire'])

## Forecasting an individual region.

Use the `.predict()` method to make a forecast.  The method takes 3 parameters:

* **horizon**: int - the forecast horizon e.g. 84 days
* **alpha**: float, optional (default=0.2) - a value between 0 and 1 and used to construct a 100(1 - alpha) prediction interval. E.g. alpha=0.2 returns a 80\% interval.  
* **return_all_models**: bool, optional (default=False). If sets to true returns the ensemble prediction AND the Prophet and Regression predictions.

In [8]:
#example 1: predict 5 days ahead - remember we have fitted Wiltshire training data.
forecast_frame = model.predict(horizon=5)
forecast_frame

,yhat,yhat_lower_80,yhat_upper80
ds,,,
2020-01-01,388.956524,365.379215,412.670244
2020-01-02,328.510997,303.526024,351.859682
2020-01-03,332.585208,308.241158,356.702919
2020-01-04,348.749082,324.353205,372.963615
2020-01-05,348.874887,324.847488,373.788973


The method returns a `pd.DataFrame` containing mean forecast (yhat) and an upper and lower prediction interval.  The code below demonstrates how to return predictions from both the ARIMA and Prophet models.  We will also return a different prediction interval.

In [9]:
#example 2: predict 5 days ahead, return 95% PI and individual model preds
forecast_frame = model.predict(horizon=5, alpha=0.05, return_all_models=True)
forecast_frame

,yhat,yhat_lower_95,yhat_upper95,arima_mean,arima_lower_95,arima_upper_95,prophet_mean,prophet_lower_95,prophet_upper_95
ds,,,,,,,,,
2020-01-01,388.956524,354.060916,425.425824,381.658539,345.203750,418.113328,396.254509,362.918083,432.738320
2020-01-02,328.510997,292.072902,364.647750,328.900688,291.654361,366.147014,328.121306,292.491443,363.148486
2020-01-03,332.585208,295.614891,369.075244,333.121287,295.435440,370.807135,332.049130,295.794341,367.343353
2020-01-04,348.749082,311.613435,386.125793,347.825116,309.739809,385.910423,349.673048,313.487061,386.341162
2020-01-05,348.874887,311.404883,386.414107,348.558039,310.108518,387.007559,349.191736,312.701247,385.820655


## An 'all in one' forecast function

As an alternative to the above the `utility` module contains a convenience function called `forecast`.  This is an all-in-one function.  Just pass in your training data (for a single time series) and horizon.

In [10]:
forecast(clean['Wiltshire'], 
         horizon=6, 
         alpha=0.2,
         return_all_models=False)

,yhat,yhat_lower_80,yhat_upper80
ds,,,
2020-01-01,388.956524,364.699238,413.315035
2020-01-02,328.510997,304.246623,352.477100
2020-01-03,332.585208,307.864453,356.804316
2020-01-04,348.749082,324.411811,373.970055
2020-01-05,348.874887,324.671761,373.882124
2020-01-06,335.659634,310.824663,360.805672


## Forecasting multiple regions in one go.

If there are multiple regions to forecast put all of the training data into the same frame (see `clean`) and pass this to the `multi_region_forecast()` function from the `utility` module.

This is an efficient function as it runs the forecasts in parrallel across your CPU cores.  E.g. if you have a 4 cores then 4 regions will be forecast simultaneously.  This will reduce model run time (assuming you have more than one Core).

In [11]:
#note depending on your machine this will take 20 seconds to run.
forecasts = multi_region_forecast(y_train=clean, horizon=8)

In [14]:
#the function returns a list of pd.DataFrame's
type(forecasts)

list

In [15]:
#results for BNSSG
forecasts[0]

,yhat,yhat_lower_80,yhat_upper80
ds,,,
2020-01-01,667.020300,637.220999,699.309558
2020-01-02,545.942158,513.427402,577.781513
2020-01-03,553.808267,521.797127,585.833330
2020-01-04,587.995970,557.017993,619.951215
2020-01-05,579.726243,548.184616,612.967722
2020-01-06,554.845507,522.637830,586.754980
2020-01-07,537.701526,504.058710,570.173386
2020-01-08,534.316426,501.323339,566.536848


In [16]:
#results for Cornwall is at index 1 etc.
forecasts[1]

,yhat,yhat_lower_80,yhat_upper80
ds,,,
2020-01-01,318.257114,296.902681,339.158512
2020-01-02,252.668308,231.636558,273.874112
2020-01-03,255.642661,234.589574,277.186718
2020-01-04,271.156644,249.309335,292.714249
2020-01-05,273.986725,252.509054,296.290844
2020-01-06,261.516648,239.797063,282.910243
2020-01-07,250.705678,229.066370,272.699642
2020-01-08,248.799221,226.760647,270.244500
